In [47]:
#install the Pyterrier framework
!pip install python-terrier
# install the nltk modules
!pip install nltk

In [48]:
# Need to install additional terrier package for PRF. It will take around 1 min
!git clone https://github.com/terrierteam/terrier-prf/
!apt-get install maven   #used for Java projects to manage project dependencies and build processes
%cd /content/terrier-prf/
!mvn install
!pwd
%cd ..

fatal: destination path 'terrier-prf' already exists and is not an empty directory.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
maven is already the newest version (3.6.3-5).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
/content/terrier-prf
[INFO] Scanning for projects...
[INFO] 
[INFO] ----------------------< org.terrier:terrier-prf >-----------------------
[INFO] Building terrier-prf 0.2-SNAPSHOT
[INFO] --------------------------------[ jar ]---------------------------------
[INFO] 
[INFO] --- maven-resources-plugin:2.6:resources (default-resources) @ terrier-prf ---
[INFO] Using 'UTF-8' encoding to copy filtered resources.
[INFO] skip non existing resourceDirectory /content/terrier-prf/src/main/resources
[INFO] 
[INFO] --- maven-compiler-plugin:3.1:compile (default-compile) @ terrier-prf ---
[INFO] Nothing to compile - all classes are up to date
[INFO] 
[INFO] --- maven-resources-plugin:2.6:testResources (defaul

In [49]:
import pyterrier as pt
if not pt.started():
  # In this lab, we need to specify that we start PyTerrier with PRF enabled
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

In [50]:
#Import the necessary modules:
import ipywidgets as widgets
from IPython.display import display
import pandas as pd
import pyterrier as pt
import time
import pandas as pd
import pyterrier as pt
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import *
from nltk.stem.porter import *
import re # used to clean the dat
import os
#to display the full text on the notebook without truncation
pd.set_option('display.max_colwidth', 150)

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [51]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [52]:
# Initialize Porter stemmer
stemmer = PorterStemmer()

Data Collection


In [53]:
vaswani_dataset = pt.datasets.get_dataset("vaswani")

In [54]:
topic = vaswani_dataset.get_topics()
topic.head()

,qid,query
0,1,measurement of dielectric constant of liquids by the use of microwave techniques
1,2,mathematical analysis and design details of waveguide fed microwave radiations
2,3,use of digital computers in the design of band pass filters having given phase and attenuation characteristics
3,4,systems of data coding for information transfer
4,5,use of programs in engineering testing of computers


In [55]:
qrels = vaswani_dataset.get_qrels()
qrels.head()

,qid,docno,label
0,1,1239,1
1,1,1502,1
2,1,4462,1
3,1,4569,1
4,1,5472,1


In [56]:
topic["docno"]=topic["qid"].astype(str)
qrels['docno']=qrels['docno'].astype(str)

In [57]:
topic

,qid,query,docno
0,1,measurement of dielectric constant of liquids by the use of microwave techniques,1
1,2,mathematical analysis and design details of waveguide fed microwave radiations,2
2,3,use of digital computers in the design of band pass filters having given phase and attenuation characteristics,3
3,4,systems of data coding for information transfer,4
4,5,use of programs in engineering testing of computers,5
...,...,...,...
88,89,tunnel diode construction and its electrical characteristics explained,89
89,90,electronic density of states at the surface of a semiconductor compared with that at depth,90
90,91,resistivity of metallic thin films related to surface roughness,91
91,92,the phenomenon of radiation caused by charged particles moving in varying electric and magnetic fields,92


In [58]:
qrels

,qid,docno,label
0,1,1239,1
1,1,1502,1
2,1,4462,1
3,1,4569,1
4,1,5472,1
...,...,...,...
2078,93,9875,1
2079,93,9956,1
2080,93,10497,1
2081,93,11191,1


In [59]:
def tokenizetion(text):
  tokens = word_tokenize(text)
  return tokens

In [60]:
def lowercasing(tokens):
    lowercase_tokens = [word.lower() for word in tokens]
    return lowercase_tokens

In [61]:
def stopword_removal(tokens):
    filtered_tokens = [ word for word in tokens if word not in stop_words]
    return filtered_tokens

In [62]:
def Steem_text(text):
    stemmed_tokens = [stemmer.stem(word) for word in text]
    return stemmed_tokens

In [63]:
def preprocess(sentence):
  tokens = tokenizetion(sentence)
  lowercase_tokens = lowercasing(tokens)
  filtered_tokens = stopword_removal(lowercase_tokens)
  stemmed_tokens = Steem_text(filtered_tokens)
  sentence = ' '.join(stemmed_tokens)
  # print(sentence)
  return sentence

In [64]:
#applying preprocessing
topic['preprocessed_query'] = topic['query'].apply(preprocess)
topic.head()

,qid,query,docno,preprocessed_query
0,1,measurement of dielectric constant of liquids by the use of microwave techniques,1,measur dielectr constant liquid use microwav techniqu
1,2,mathematical analysis and design details of waveguide fed microwave radiations,2,mathemat analysi design detail waveguid fed microwav radiat
2,3,use of digital computers in the design of band pass filters having given phase and attenuation characteristics,3,use digit comput design band pass filter given phase attenu characterist
3,4,systems of data coding for information transfer,4,system data code inform transfer
4,5,use of programs in engineering testing of computers,5,use program engin test comput


Indexing

In [65]:
indexref = vaswani_dataset.get_index()
index = pt.IndexFactory.of(indexref)

print(index.getCollectionStatistics().toString())

Number of documents: 11429
Number of terms: 7756
Number of postings: 224573
Number of fields: 1
Number of tokens: 271581
Field names: [text]
Positions:   false



In [66]:
for kv in index.getLexicon():
  print("%s -> %s " % (kv.getKey(), kv.getValue().toString()))

Streaming output truncated to the last 5000 lines.
graphit -> term4484 Nt=3 TF=3 maxTF=1 @{0 149693 4} TFf=3 
grapho -> term3939 Nt=1 TF=1 maxTF=1 @{0 149702 6} TFf=1 
grasp -> term6065 Nt=1 TF=1 maxTF=1 @{0 149706 0} TFf=1 
grate -> term607 Nt=14 TF=27 maxTF=6 @{0 149709 4} TFf=27 
graupel -> term4749 Nt=1 TF=1 maxTF=1 @{0 149746 4} TFf=1 
gravit -> term1552 Nt=40 TF=52 maxTF=3 @{0 149750 0} TFf=52 
graviti -> term2287 Nt=9 TF=9 maxTF=1 @{0 149837 4} TFf=9 
graybeal -> term5582 Nt=1 TF=1 maxTF=1 @{0 149860 6} TFf=1 
graz -> term7001 Nt=1 TF=1 maxTF=1 @{0 149864 2} TFf=1 
graze -> term580 Nt=4 TF=4 maxTF=1 @{0 149868 0} TFf=4 
greas -> term6596 Nt=1 TF=1 maxTF=1 @{0 149877 2} TFf=1 
great -> term1667 Nt=49 TF=52 maxTF=3 @{0 149880 6} TFf=52 
greater -> term87 Nt=161 TF=165 maxTF=2 @{0 149983 2} TFf=165 
greatest -> term2555 Nt=33 TF=35 maxTF=2 @{0 150260 4} TFf=35 
greatli -> term1056 Nt=27 TF=27 maxTF=1 @{0 150332 4} TFf=27 
grebenkemp -> term6730 Nt=1 TF=1 maxTF=1 @{0 150395 6} TFf=1

Query Processing

In [67]:
TF = pt.BatchRetrieve(index, wmodel="TF_IDF",num_results=10)
def query_processing(query):
    query = preprocess(query)
    results = TF.search(query)
    return results

Query Expansion

In [68]:
rm3_expander = pt.rewrite.RM3(index,fb_terms=10, fb_docs=100)

def expantion(query):
    rm3_qe = TF >> rm3_expander
    expanded_query = rm3_qe.search(query).iloc[0]["query"]
    return expanded_query

In [69]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np

In [70]:
elmo = hub.load("https://tfhub.dev/google/elmo/3")
def elmofun(query):
    queryl = []
    queryl.append(query)
    query_tensor = tf.convert_to_tensor(queryl, dtype=tf.string)
    embeddings = elmo.signatures["default"](query_tensor)["elmo"]

    return embeddings

user Interface

In [71]:
query = input("Enter your query: ")
results = query_processing(query)
expanded_query = expantion(query)
results_expanded = query_processing(expanded_query)
embeddings = elmofun(query)
print("expanded results:",results_expanded[['docid','score']][0:5].add_suffix('_2'))
print(f'result: {results}')
print(f'embeddings: {embeddings}')

Enter your query: measure
expanded results:    docid_2   score_2
0     9619  3.577422
1     5417  3.517304
2     1116  3.483765
3    10986  3.447322
4    11108  3.439644
result:   qid  docid docno  rank     score   query
0   1   3012  3013     0  3.103851  measur
1   1   8754  8755     1  3.074043  measur
2   1   3180  3181     2  2.975172  measur
3   1   5417  5418     3  2.975172  measur
4   1   7439  7440     4  2.975172  measur
5   1   9619  9620     5  2.975172  measur
6   1   9559  9560     6  2.906564  measur
7   1   1268  1269     7  2.894464  measur
8   1   1805  1806     8  2.894464  measur
9   1   3013  3014     9  2.894464  measur
embeddings: [[[-0.2418702   0.25662708  0.5449727  ...  0.46025118  0.0158349
    0.13344985]]]
